In [1]:
!pip install transformers sentence-transformers langchain langchain-community langchain-openai faiss-cpu unstructured unstructured[pdf]

In [14]:
import  os
import pytesseract
# set the tesseract path for pytesseract
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

from langchain_openai import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA


In [3]:
os.environ["OPENAI_API_KEY"] = ""

In [4]:
llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [18]:
!apt-get install poppler-utils
!apt-get install tesseract-ocr
!pip install pytesseract
!pip install nltk
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.6).
0 upgraded, 0 newly installed, 0 to remove and 55 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 55 not upgraded.


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [19]:
# load the document
loader = UnstructuredFileLoader("/content/all.pdf")
documents = loader.load()

In [20]:
type(documents[0])

langchain_core.documents.base.Document

In [21]:
# create text chunks

text_splitter = CharacterTextSplitter(separator='/n',
                                      chunk_size=1000,
                                      chunk_overlap=200)

text_chunks = text_splitter.split_documents(documents)

In [22]:
# loading the vector embedding model
embeddings = HuggingFaceEmbeddings()
# vector embedding for text chunks
knowledge_base = FAISS.from_documents(text_chunks, embeddings)

<ipython-input-22-81e3a0caf0fc>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
<ipython-input-22-81e3a0caf0fc>:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggin

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [23]:
# chain for aq retrieval
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=knowledge_base.as_retriever())

In [24]:
question = "What is this document about?"
response = qa_chain.invoke({"query": question})
print(response["result"])

The document is a research paper titled "Attention Is All You Need," which introduces the Transformer model, a novel neural network architecture for sequence transduction tasks such as machine translation. The Transformer model relies entirely on attention mechanisms, eliminating the need for recurrent or convolutional neural networks. The paper discusses the model's architecture, including its encoder-decoder structure, self-attention mechanisms, and multi-head attention. It also presents experimental results demonstrating the model's superior performance and efficiency compared to previous state-of-the-art models in machine translation tasks.


In [25]:
response

{'query': 'What is this document about?',
 'result': 'The document is a research paper titled "Attention Is All You Need," which introduces the Transformer model, a novel neural network architecture for sequence transduction tasks such as machine translation. The Transformer model relies entirely on attention mechanisms, eliminating the need for recurrent or convolutional neural networks. The paper discusses the model\'s architecture, including its encoder-decoder structure, self-attention mechanisms, and multi-head attention. It also presents experimental results demonstrating the model\'s superior performance and efficiency compared to previous state-of-the-art models in machine translation tasks.'}

In [26]:
question = "What is the model architecture discussed in this paper?"
response = qa_chain.invoke({"query": question})
response["result"]

"The model architecture discussed in this paper is the Transformer. It is based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. The Transformer uses stacked self-attention and point-wise, fully connected layers for both the encoder and decoder. The encoder is composed of a stack of identical layers, each with a multi-head self-attention mechanism and a fully connected feed-forward network. The decoder is similar but includes an additional sub-layer for multi-head attention over the encoder's output. This architecture allows for significant parallelization and improved translation quality."

In [ ]:
question = "What are the applications of attention in our Model?"
response = qa_chain.invoke({"query": question})
response["result"]

'The Transformer model uses multi-head attention in three different ways:\n\n1. **Encoder-Decoder Attention**: In these layers, the queries come from the previous decoder layer, and the memory keys and values come from the output of the encoder. This allows every position in the decoder to attend over all positions in the input sequence. This mechanism mimics the typical encoder-decoder attention mechanisms in sequence-to-sequence models.\n\n2. **Self-Attention in the Encoder**: In these layers, all of the keys, values, and queries come from the same place, specifically the output of the previous layer in the encoder. Each position in the encoder can attend to all positions in the previous layer of the encoder.\n\n3. **Self-Attention in the Decoder**: Similar to the encoder, these layers allow each position in the decoder to attend to all positions in the decoder up to and including that position. To preserve the auto-regressive property, the model prevents positions from attending to 

In [ ]:
print(response["result"])

The Transformer model uses multi-head attention in three different ways:

1. **Encoder-Decoder Attention**: In these layers, the queries come from the previous decoder layer, and the memory keys and values come from the output of the encoder. This allows every position in the decoder to attend over all positions in the input sequence. This mechanism mimics the typical encoder-decoder attention mechanisms in sequence-to-sequence models.

2. **Self-Attention in the Encoder**: In these layers, all of the keys, values, and queries come from the same place, specifically the output of the previous layer in the encoder. Each position in the encoder can attend to all positions in the previous layer of the encoder.

3. **Self-Attention in the Decoder**: Similar to the encoder, these layers allow each position in the decoder to attend to all positions in the decoder up to and including that position. To preserve the auto-regressive property, the model prevents positions from attending to subsequ